In [8]:
from langchain_community.document_loaders import TextLoader
from  langchain_text_splitters import CharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma

In [10]:
%pip install python-dotenv langchain-community langchain-opencv langchain-chroma gradio jupyterlab ipywidgets langchain-openai


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
%pip install langchain sentence-transformers



[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd
books = pd.read_csv("books_cleaned.csv")

In [13]:
books

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing description,age_of_book,words_in_description,title_and_subtitle,tagged_description
0,9780002005883,0002005883,Gilead,NaN,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,361.0,0,20.0,199,Gilead,9780002005883 A NOVEL THAT READERS and critics...
1,9780002261982,0002261987,Spider's Web,A Novel,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,5164.0,0,24.0,205,Spider's Web: A Novel,9780002261982 A new 'Christie for Christmas' -...
2,9780006178736,0006178731,Rage of angels,NaN,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,29532.0,0,31.0,57,Rage of angels,"9780006178736 A memorable, mesmerizing heroine..."
3,9780006280897,0006280897,The Four Loves,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,33684.0,0,22.0,45,The Four Loves,9780006280897 Lewis' work on the nature of lov...
4,9780006280934,0006280935,The Problem of Pain,NaN,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,37569.0,0,22.0,75,The Problem of Pain,"9780006280934 ""In The Problem of Pain, C.S. Le..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5398,9788172235222,8172235224,Mistaken Identity,NaN,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,0.0,0,21.0,288,Mistaken Identity,9788172235222 On A Train Journey Home To North...
5399,9788173031014,8173031010,Journey to the East,NaN,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,24.0,0,22.0,63,Journey to the East,9788173031014 This book tells the tale of a ma...
5400,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,NaN,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,1568.0,0,21.0,117,The Monk Who Sold His Ferrari: A Fable About F...,9788179921623 Wisdom to Create a Life of Passi...
5401,9788185300535,8185300534,I Am that,Talks with Sri Nisargadatta Maharaj,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,104.0,0,25.0,174,I Am that: Talks with Sri Nisargadatta Maharaj,9788185300535 This collection of the timeless ...


In [14]:
books["tagged_description"].to_csv("tagged_description.txt",
                                   index=False,
                                   header=False)

In [15]:
raw_documents = TextLoader("tagged_description.txt").load()
text_splitter = CharacterTextSplitter(chunk_size=0, chunk_overlap=0, separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 1170, which is longer than the specified 0
Created a chunk of size 1216, which is longer than the specified 0
Created a chunk of size 375, which is longer than the specified 0
Created a chunk of size 311, which is longer than the specified 0
Created a chunk of size 483, which is longer than the specified 0
Created a chunk of size 484, which is longer than the specified 0
Created a chunk of size 962, which is longer than the specified 0
Created a chunk of size 188, which is longer than the specified 0
Created a chunk of size 845, which is longer than the specified 0
Created a chunk of size 296, which is longer than the specified 0
Created a chunk of size 197, which is longer than the specified 0
Created a chunk of size 881, which is longer than the specified 0
Created a chunk of size 1090, which is longer than the specified 0
Created a chunk of size 1191, which is longer than the specified 0
Created a chunk of size 306, which is longer than the specified 0
Create

Created a chunk of size 421, which is longer than the specified 0
Created a chunk of size 889, which is longer than the specified 0
Created a chunk of size 695, which is longer than the specified 0
Created a chunk of size 304, which is longer than the specified 0
Created a chunk of size 219, which is longer than the specified 0
Created a chunk of size 277, which is longer than the specified 0
Created a chunk of size 146, which is longer than the specified 0
Created a chunk of size 2010, which is longer than the specified 0
Created a chunk of size 175, which is longer than the specified 0
Created a chunk of size 481, which is longer than the specified 0
Created a chunk of size 302, which is longer than the specified 0
Created a chunk of size 522, which is longer than the specified 0
Created a chunk of size 649, which is longer than the specified 0
Created a chunk of size 778, which is longer than the specified 0
Created a chunk of size 165, which is longer than the specified 0
Created a

In [20]:
documents[0]

Document(metadata={'source': 'tagged_description.txt'}, page_content='"9780002005883 A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, G

In [17]:
%pip install -U langchain-huggingface


[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [18]:
from langchain_huggingface import HuggingFaceEmbeddings


embed_model = HuggingFaceEmbeddings(
    model_name="/Users/parth/Desktop/all-mpnet-base-v2",
    encode_kwargs={"batch_size": 16}
)


In [19]:
from langchain_chroma import Chroma

db = Chroma.from_documents(
    documents=documents,
    embedding=embed_model,
    persist_directory="chroma_db"  
)

In [21]:
test = embed_model.embed_query("test string")
print(len(test))


768


In [22]:
query = "A book to teach children about nature"
docs = db.similarity_search(query, k = 10)
docs



[Document(id='a26f8e02-db69-4b76-a893-c4ae1305aa58', metadata={'source': 'tagged_description.txt'}, page_content='"9780067575208 First published more than three decades ago, this reissue of Rachel Carson\'s award-winning classic brings her unique vision to a new generation of readers. Stunning new photographs by Nick Kelsh beautifully complement Carson\'s intimate account of adventures with her young nephew, Roger, as they enjoy walks along the rocky coast of Maine and through dense forests and open fields, observing wildlife, strange plants, moonlight and storm clouds, and listening to the ""living music"" of insects in the underbrush. ""If a child is to keep alive his inborn sense of wonder."" Writes Carson, ""he needs the companionship of at least one adult who can share it, rediscovering with him the joy, excitement and mystery of the world we live in."" The Sense of Wonder is a refreshing antidote to indifference and a guide to capturing the simple power of discovery that Carson v

In [23]:


def retrieve_semantic_recommendations(
        query: str,
        top_k: int = 10,
) -> pd.DataFrame:
    recs = db.similarity_search(query, k = 50)

    books_list = []

    for i in range(0, len(recs)):
        books_list += [int(recs[i].page_content.strip('"').split()[0])]

    return books[books["isbn13"].isin(books_list)]



In [24]:
retrieve_semantic_recommendations("A book to teach children about nature")

,isbn13,isbn10,title,subtitle,authors,categories,thumbnail,description,published_year,average_rating,num_pages,ratings_count,missing description,age_of_book,words_in_description,title_and_subtitle,tagged_description
31,9780006754893,0006754899,A Small Pinch of Weather,And Other Stories,Joan Aiken,"Children's stories, English",http://books.google.com/books/content?id=QiFhO...,A magical and fantastic collection of early st...,2000.0,4.27,239.0,116.0,0,24.0,24,A Small Pinch of Weather: And Other Stories,9780006754893 A magical and fantastic collecti...
61,9780007151240,0007151241,The Family Way,NaN,Tony Parsons,Parenthood,http://books.google.com/books/content?id=dJEIx...,It should be the most natural thing in the wor...,2005.0,3.51,400.0,2095.0,0,19.0,29,The Family Way,9780007151240 It should be the most natural th...
146,9780060546571,0060546573,Three Rotten Eggs,NaN,Gregory Maguire,Juvenile Fiction,http://books.google.com/books/content?id=t2pWl...,The students of Miss Earth's class in rural Ve...,2005.0,3.74,240.0,76.0,0,19.0,30,Three Rotten Eggs,9780060546571 The students of Miss Earth's cla...
149,9780060556501,0060556501,"The Lion, the Witch and the Wardrobe (picture ...",NaN,C. S. Lewis,Juvenile Fiction,http://books.google.com/books/content?id=FlSpp...,Narnia: A magical land full of wonder and exci...,2004.0,4.19,48.0,5012.0,0,20.0,172,"The Lion, the Witch and the Wardrobe (picture ...",9780060556501 Narnia: A magical land full of w...
259,9780060845247,0060845244,"The Lion, the Witch and the Wardrobe Read-Alou...",NaN,C. S. Lewis;Pauline Baynes,Juvenile Fiction,http://books.google.com/books/content?id=deiyG...,Four English school children enter the magic l...,2005.0,4.21,189.0,56.0,0,19.0,35,"The Lion, the Witch and the Wardrobe Read-Alou...",9780060845247 Four English school children ent...
329,9780060959036,0060959037,Prodigal Summer,A Novel,Barbara Kingsolver,Fiction,http://books.google.com/books/content?id=06IwG...,Barbara Kingsolver's fifth novel is a hymn to ...,2001.0,4.00,444.0,85440.0,0,23.0,76,Prodigal Summer: A Novel,9780060959036 Barbara Kingsolver's fifth novel...
380,9780061131622,0061131628,Mandy,NaN,Julie Andrews Edwards,Juvenile Fiction,http://books.google.com/books/content?id=5dNj_...,"Mandy, a ten-year-old orphan, dreams of a plac...",2006.0,4.24,320.0,9482.0,0,18.0,107,Mandy,"9780061131622 Mandy, a ten-year-old orphan, dr..."
413,9780064402453,0064402452,Racso and the Rats of NIMH,NaN,Jane Leslie Conly,Juvenile Fiction,http://books.google.com/books/content?id=MgoNv...,"‘Racso, a brash and boastful little rodent, is...",1988.0,3.76,288.0,3231.0,0,36.0,98,Racso and the Rats of NIMH,"9780064402453 ‘Racso, a brash and boastful lit..."
431,9780064409902,0064409902,Old Town in the Green Groves,Laura Ingalls Wilder's Lost Little House Years,Cynthia Rylant;Jim LaMarche,Juvenile Fiction,http://books.google.com/books/content?id=YUQMA...,"After grasshoppers ruin the crops, eight-year-...",2004.0,4.04,176.0,4631.0,0,20.0,34,Old Town in the Green Groves: Laura Ingalls Wi...,9780064409902 After grasshoppers ruin the crop...
437,9780064434874,0064434877,Christmas in the Big Woods,NaN,Laura Ingalls Wilder,Juvenile Fiction,http://books.google.com/books/content?id=FT1Yp...,"Long ago, a little girl named Laura Ingalls li...",1997.0,4.19,32.0,2062.0,0,27.0,145,Christmas in the Big Woods,"9780064434874 Long ago, a little girl named La..."
